In [28]:
import rootutils

current_folder = globals()['_dh'][0]

rootutils.setup_root(current_folder, indicator=".project-root", pythonpath=True)

WindowsPath('C:/Users/vlad1/GitHub/TopoHyperDrive')

In [101]:
experimets = {
    "TPESampler": "../logs/tpe_search/multiruns/2024-05-22_17-44-08",
    "RandomSearch": "../logs/random_search/multiruns/2024-05-22_14-40-01",
    "One-to-Random": "../logs/one_to_random_single_objective_search/multiruns/2024-05-23_21-40-16",
    "One-to-Random-Multi": "../logs/one_to_random_multi_objective_search/multiruns/2024-05-24_10-40-54",
    "PCA One-to-Random-Multi": "../logs/pca_one_to_random_multi_objective_search/multiruns/2024-05-24_15-32-36",
}

In [112]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker


def save_formatted(fig, ax, settings, save_path, xlabel=None, ylabel=None, title=None):
    """
    Saves (fig, ax) object to save_path with settigns json file
    """
    # labels and title
    if settings["show labels"]:
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
    else:
        ax.set_xlabel(None)
        ax.set_ylabel(None)
    if settings["show title"]:
        plt.title(title)
    if settings["font size"]:
        plt.rcParams.update({'font.size': settings["font size"]})
    #
    ax.tick_params(
        axis="both", which="major", labelsize=settings["tick labels font size"]
    )
    ax.tick_params(
        axis="both", which="minor", labelsize=2 * settings["tick labels font size"] // 3
    )
    # set height and width
    fig.set_figheight(settings["fig height"])
    fig.set_figwidth(settings["fig width"])
    # set aspect ratio
    x0, x1 = ax.get_xlim()
    y0, y1 = ax.get_ylim()
    ax.set_aspect(settings["aspect ratio"] * abs((x1 - x0) / (y1 - y0)))
    # save
    plt.savefig(save_path, dpi=settings["dpi"], bbox_inches="tight")
    plt.clf()


def extract_best_val_accuracy(path: str, rtd: bool = False):
    experiments = sorted(filter(lambda x: x.isdigit(), os.listdir(path)), key=lambda x: int(x))
    
    best_val_accuracies = []
    test_accuracies = []
    steps = [0]
    rtds = []
    for experiment in experiments:
        try:
            metrics = pd.read_csv(os.path.join(path, experiment, "csv/version_0/metrics.csv"))
            new_step = steps[-1] + metrics["step"].iloc[-1]//1000
            metrics_val = metrics["val/acc_best"]
            metrics_test = metrics["test/accuracy"]
            best_val_accuracies.append(metrics_val.loc[metrics_val.last_valid_index()])
            test_accuracies.append(metrics_test.loc[metrics_test.last_valid_index()])
            steps.append(new_step)
            if rtd:
                rtds.append(metrics["RTD_score"].iloc[-1])
        except:
            continue
        
    current_best = -1
    current_val = -1
    current_test = -1
    hp_search_accuracies = []
    hp_search_test_accuracies = []
    if not rtd:
        for best_val_accuracy, test_accuracy in zip(best_val_accuracies, test_accuracies):
            if best_val_accuracy > current_best:
                current_best = best_val_accuracy
                current_test = test_accuracy
            hp_search_accuracies.append(current_best)
            hp_search_test_accuracies.append(current_test)
    else:
        for rtd_val, best_val_accuracy, test_accuracy in zip(rtds, best_val_accuracies, test_accuracies):
            if rtd_val > current_best:
                current_best = rtd_val
                current_val = best_val_accuracy
                current_test = test_accuracy
            hp_search_accuracies.append(current_val)
            hp_search_test_accuracies.append(current_test)
    
    return steps[1:], hp_search_accuracies, hp_search_test_accuracies

def plot_experiments(
    experiments: dict,
    settings: dict,
    stage: str
):
    fig, axs = plt.subplots()
    formatter = ticker.ScalarFormatter(useMathText=True)
    formatter.set_scientific(True)
    formatter.set_powerlimits((-2, 2))
    axs.yaxis.set_major_formatter(formatter)
    
    for experiment in experiments:
        steps, hp_search_accuracies, hp_search_test_accuracies = extract_best_val_accuracy(experimets[experiment], experiment=="One-to-Random")
        if stage == "val":
            axs.plot(steps, hp_search_accuracies, label=experiment)
        else:
            axs.plot(steps, hp_search_test_accuracies, label=experiment)
    axs.legend()
    save_formatted(
        fig,
        axs,
        settings,
        save_path=f"../paper/accuracy_{stage}.pdf",
        xlabel="Steps, $10^3$",
        ylabel="Accuracy",
        title="Best validation accuracy" if stage == "val" else "Test accuracy",
    )
    

In [113]:
settings = {
    "aspect ratio": 1.00,
    "dpi": 400,
    "fig height": 7,
    "fig width": 5,
    "tick labels font size": 15,
    "style": "classic",
    "show labels": True,
    "show title": True,
    "font size": 15
}

In [114]:
plot_experiments(experimets, settings, "val")

<Figure size 500x700 with 0 Axes>

In [115]:
plot_experiments(experimets, settings, "test")

<Figure size 500x700 with 0 Axes>

In [ ]:
from hydra_plugins.hydra_optuna_sweeper.optuna_sweeper import OptunaSweeper